# Inference for greyscale

    copying the same flow as the training flow but for inference

In [ ]:
from __future__ import print_function
import tensorflow as tf
import numpy as np

import TensorflowUtils as utils
import read_MITSceneParsingData as scene_parsing
import datetime
import DatasetReader as dataset
from six.moves import xrange
from scipy.io import loadmat

import scipy as scp
import scipy.misc as misc
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
%matplotlib inline
import random
import os
from PIL import Image

from keras import backend as K

from keras.models import Sequential, Model
from keras.layers import Input, Dropout, Permute, Add, add
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D, Deconvolution2D, Cropping2D
from keras.layers import InputLayer

from fcn_keras2 import fcn32_blank, fcn32_blank_greyscale, fcn_32s_to_16s

############ Declarations  ##############



MODEL_URL = 'http://www.vlfeat.org/matconvnet/models/beta16/imagenet-vgg-verydeep-19.mat'
DATA_URL = './Data_zoo/Test.zip'

NUM_OF_CLASSESS = 12

BATCH_SIZE = 1
IMAGE_SIZE = 512
LEARNING_RATE = 1e-4
MAX_ITERATION = int(1e5 + 1)
DEBUG = True

LOGS_DIR = "logs5/"
DATA_DIR = "Data_zoo/"
LEARNING_RATE = 1e-4
MODEL_DIR ="Model_zoo/"
DATA_URL = './Data_zoo/Test.zip'

def inference(image):

    fcn32model = fcn32_blank_greyscale(IMAGE_SIZE)

    fcn16model = fcn_32s_to_16s(fcn32model)

    data = loadmat('pascal-fcn16s-dag.mat', matlab_compatible=False, struct_as_record=False)
    l = data['layers']
    p = data['params']
    description = data['meta'][0,0].classes[0,0].description

    class2index = {}
    for i, clname in enumerate(description[0,:]):
        class2index[str(clname[0])] = i

    logits = fcn16model(image)

    annotation_pred = tf.argmax(logits, dimension=3, name="prediction")

    return tf.expand_dims(annotation_pred, dim=3), logits


def train(loss_val, var_list):
    optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
    grads = optimizer.compute_gradients(loss_val, var_list=var_list)

    return optimizer.apply_gradients(grads)


keep_probability = tf.placeholder(tf.float32, name="keep_probabilty")
image = tf.placeholder(tf.float32, shape=[None, IMAGE_SIZE, IMAGE_SIZE, 1], name="input_image")
annotation = tf.placeholder(tf.int32, shape=[None, IMAGE_SIZE, IMAGE_SIZE, 1], name="annotation")

pred_annotation, logits = inference(image)


loss = tf.reduce_mean((tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=tf.squeeze(annotation, 
                                                                    squeeze_dims=[3]), name="entropy")))

print("types on prediction tensors:  ", type(annotation), type(pred_annotation) )

# Accuracy
correct_prediction = tf.equal(tf.argmax(annotation,3), tf.argmax(pred_annotation,3))
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


trainable_var = tf.trainable_variables()

train_op = train(loss, trainable_var)


print("Setting up dataset reader")
image_options = {'resize': True, 'resize_size': IMAGE_SIZE}

validation_dataset_reader = dataset.Dataset_evo_interactive(IMAGE_SIZE, sample_size=200, shape_count = 20, r_min=1, r_max=10)

sess = tf.Session()
K.set_session(sess)

print("Setting up Saver...")
saver = tf.train.Saver()
summary_writer = tf.summary.FileWriter(LOGS_DIR, sess.graph)

sess.run(tf.global_variables_initializer())
ckpt = tf.train.get_checkpoint_state(LOGS_DIR)
if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path)
    print("Model restored...")



In [ ]:
def color_image(image, num_classes=20):
    import matplotlib as mpl
    import matplotlib.cm
    norm = mpl.colors.Normalize(vmin=0., vmax=num_classes)
    mycm = mpl.cm.get_cmap('Set3')
    return mycm(norm(image))

In [ ]:
def predict_and_display(valid_images, valid_annotations, pred, NUM_OF_CLASSESS):
    
    unique, counts = np.unique(valid_annotations, return_counts=True)
    for n in range(len(unique)):
        print(unique[n], counts[n])

    fig = plt.figure()
    fig.set_size_inches(18, 18)

    

    patch = fig.add_subplot(1,3,1)
    patch.axes.get_xaxis().set_visible(False)
    patch.axes.get_yaxis().set_visible(False)
    patch.imshow(valid_images[0]);


    new_image = color_image(valid_annotations[0], NUM_OF_CLASSESS)
    patch = fig.add_subplot(1,3,2)
    patch.axes.get_xaxis().set_visible(False)
    patch.axes.get_yaxis().set_visible(False)
    patch.imshow(new_image);
    
    new_image1 = color_image(pred[0], NUM_OF_CLASSESS)
    patch = fig.add_subplot(1,3,3)
    patch.axes.get_xaxis().set_visible(False)
    patch.axes.get_yaxis().set_visible(False)
    patch.imshow(new_image1);


In [ ]:
validation_dataset_reader = dataset.Dataset_evo_interactive(IMAGE_SIZE, sample_size=100, shape_count = 1000, r_min=1, r_max=5)


In [ ]:
# run the generator

batch_size = 1

valid_images, valid_annotations = validation_dataset_reader.next_batch(1)
print(valid_images.shape)
print(valid_annotations.shape)

pred = sess.run(pred_annotation, feed_dict={image: valid_images, keep_probability: 1.0})
print(pred.shape)

predict_and_display(np.squeeze(valid_images,axis=3), np.squeeze(valid_annotations, axis=3),np.squeeze(pred,axis=3), 7)

In [ ]:
comp_images, comp_annotations = validation_dataset_reader.next_batch(1)

In [ ]:
print(comp_images.shape)

In [ ]:
pred = sess.run(pred_annotation, feed_dict={image: comp_images, keep_probability: 1.0})
print(pred.shape)

predict_and_display(np.squeeze(comp_images,axis=3), np.squeeze(comp_annotations, axis=3),np.squeeze(pred,axis=3), 7)

In [ ]:
np.save('image_test', comp_images)
np.save('annotation_test', comp_annotations)